# LDA
* gensim 是功能強大的套件，處理巨量文本資料特別有用。今天我要介紹 LDA, Latent Dirichlet allocation (不要跟另一個字眼混淆
* Linear Discriminant Analysis，兩者完全不同)。
* LDA 與 LSA 功能類同，都是從字詞與文章間抽離出概念，再投射到向量空間，不過，前者是用機械學習的方法，後者則是用SVD降維的方式。
* 大數據的時代，前者的使用似乎比後者更廣泛。
* 同學上課前須先安裝 gensim, smart_open 模組。

## 安裝 gensim
* gensim 官方網頁: https://radimrehurek.com/gensim/tut1.html
* 有完整範例與說明，請自行參考。
* 安裝 gensim: 
* 命令提示字元：pip install --upgrade gensim 或 easy_install -U gensim
* 安裝 smart_open: 
* 命令提示字元：pip install smart_open

* See this *gensim* tutorial on the web [here](https://radimrehurek.com/gensim/tut1.html).

In [2]:
path='C:/Users/HSIU/Desktop/108-1/python_ntu'

# 叫進所有會用到的模組

In [3]:
from __future__ import print_function
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
import pandas as pd
import numpy
import numpy as np
import matplotlib.pyplot as plt
import os
import io
import re

# 讀取文章 feature

In [4]:
#####################################################################
import pandas as pd
# path='d:/My python/'
df=pd.read_csv(path+"/Gossiping_ran2019_5000_token_feature.csv", index_col=None)
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,name,ym,title,date,month,year,push,trash,text,token,feature
0,0,0,0,mike143677 (包子大撒避),201910,[問卦] 鳳梨蝦球的鳳梨是不是超級冗員？,2019-Oct-19,Oct,2019.0,17,7,小弟超愛吃鳳梨蝦球這道菜但是絕對不吃鳳梨以往餐桌上的鳳梨蝦球也是蝦球吃光光鳳梨只能慘遭丟進廚...,小弟 超愛吃 鳳梨 蝦球 這 道菜 但是 絕對 不吃 鳳梨 以往 餐桌上 的 鳳梨 蝦球 也...,丟進 冗員 堪稱 多餘 廚餘桶 慘遭 蝦球 貝斯手 超愛吃 超級 道菜 餐桌上 鳳梨
1,1,1,1,kent21413 (kent),201910,[問卦] 慟！再一個月一年又結束了,2019-Nov-23,Nov,2019.0,4,1,是這樣的再一個月又要跨年呢好像有點快一年過去各位有甚麼還沒有完成的事情嗎剩一個月可以做甚麼大事卦,是 這樣 的 再 一個月 又 要 跨 年 呢 好像 有點 快 一年 過去 各位 有 甚麼 還...,一個月 一年 事情 可以 各位 大事 好像 完成 有點 沒有 甚麼 這樣 過去
2,2,2,2,abx310492 (胖宅男),201907,[問卦] 微軟是不是放棄IE了,2019-Aug-25,Aug,2019.0,16,1,剛電腦重灌裝完要把它設為預設瀏覽器時發現點一下就好了以前微軟還會要你確認真的不用嗎是不是被放...,剛 電腦 重 灌裝 完要 把 它 設為 預設 瀏覽器 時 發現 點 一下 就 好 了 以前 ...,一下 不用 以前 名字 完要 微軟 放棄 是不是 沒記 瀏覽器 灌裝 發現 真的 確認 設為...
3,3,3,3,Metallicat (金屬製貓),201908,[問卦] 我根本不屑中秋烤肉這種商業操作?陋習,2019-Sep-10,Sep,2019.0,11,2,中秋節是拿來賞月吃月餅柚子不知何時台灣人被烤肉醬公司給洗腦覺得中秋一定要烤肉變全民運動其實烤...,中秋節 是 拿來 賞月 吃 月餅 柚子 不知 何時 臺灣人 被 烤肉醬 公司 給 洗腦 覺得...,不屑 商業 地球 夾子 常常 木炭 洗腦 烤肉 烤肉醬 造成 陋習
4,4,4,4,IamCuteGirl (CuteGirl5566),201908,Re: [問卦] 哪邊有賣帥哥西裝啊 ☹,2019-Oct-17,Oct,2019.0,0,0,笨笨你沒看懂原意啦那個帥哥西裝是指不管高矮胖瘦身上有了變帥西裝錢大家都會尊敬喜歡驚訝暗戀從攝...,笨笨 你 沒 看 懂 原意 啦 那個 帥哥 西裝 是 指 不管 高矮 胖瘦 身上 有 了 變...,不管 動作 原意 嘻嘻 女生 寬子還 尊敬 帥哥 意思 成開 拉屎 攝影棚 暗喻 暗戀 有錢...
5,5,5,5,ccxx9999 (0-0),201910,[問卦] 真的以前有半年就考上台大甚至醫科的嗎,2019-Sep-16,Sep,2019.0,8,2,真的以前建中北一有人高中玩很瘋都沒在唸書只靠半年狂唸書就考上台大甚至醫科的嗎,真的 以前 建中 北 一 有人 高中 玩 很 瘋 都 沒 在 唸書 只靠 半年 狂 唸書 就...,以前 半年 只靠 唸書 建中 有人 甚至 真的 考上 臺大 醫科 高中
6,6,6,6,cores (24番),201910,[新聞] 談司法改革 陳師孟嘆：豬吠火車,2019-Oct-27,Oct,2019.0,6,9,談司法改革陳師孟嘆豬吠火車自由時報〔記者鍾麗華台北報導〕監委陳師孟在最新的尖尾週記表示他一個...,談 司法 改革 陳 師孟嘆 豬吠 火車 自由 時報 〔 記者 鍾 麗華 臺北 報導 〕 監委...,不料 人民 債務人 司法 司法院 在乎 大法官 尖尾 師孟 師孟說 情案 收賄 改革 法官 ...
7,7,7,7,Pcom (Pcom),201910,Re: [新聞] 「貪污說」惹怒蔡英文幕僚 柯文哲嗆：我,2019-Aug- 5,Aug,2019.0,4,2,我是這樣認為啦柯文哲本人應該到地檢署按鈴告發他所認為的每位蔡英文旁邊有貪污的人民進黨的選戰策...,我 是 這樣 認為 啦 柯文哲 本人 應該 到 地 檢署 按鈴 告發 他 所 認為 的 每位...,不法 公義 告發 情事 按鈴 柯文哲 檢署 歡迎 蔡英文 證據 貪污
8,8,8,8,Number87 (人畜無害),201911,[問卦]最近是喇牙的盛產期嗎,2019-Aug- 4,Aug,2019.0,6,2,餓死抬頭最近真的真的一直發現家裡外面裡面都有喇牙的蹤跡真的是異常的頻繁出現我知道喇牙是益蟲但...,餓死 抬頭 最近 真的 真的 一直 發現 家裡 外面 裡面 都 有 喇牙 的 蹤跡 真的 是...,出沒 喇牙 外面 天氣 太熱 家裡 專家 很少 抬頭 最好 最近 爬蟲類 異常 益蟲 看到 ...
9,9,9,9,pchunters123 (pp),201910,[新聞] 集集彩繪列車調整石虎設計 全新圖樣出爐,2019-Aug-27,Aug,2019.0,14,8,新頭殼謝佳真集集彩繪列車調整石虎設計全新圖樣出爐集集彩繪列車因為設計師江孟芝使用圖庫圖片做為...,新 頭殼 謝佳真 集集 彩繪 列車 調整 石虎 設計 全新 圖樣 出爐 集集 彩繪 列車 因...,使用 合法 嚴謹 字體 審查 康熙字典 彩繪 授權 活化 環境 石虎 管理處 觀光局 設計 ...


# 檢查並刪除空白的文章

In [5]:
# 無 feature 跳過
del_set=[]
for n in range(len(list(df.feature))):
    if type(list(df.feature)[n])!=float:
        if len(list(df.feature)[n])==0:
            del_set.append(n)      
    else:
        del_set.append(n)
print (del_set)
df.drop(axis=1, index=del_set, inplace=True)
# 重新編碼
df = df.reset_index(drop=True)
len(df)    

[]


4999

# corpus

* 製作 dictionary
* 製作 corpus

In [5]:
# 命令提示字元 pip install gensim

In [6]:
from gensim import corpora
from gensim import models
from collections import defaultdict
from gensim import similarities
from pprint import pprint 

## 讀取檔案

In [7]:
 #######################################選取文本
texts=list(df.feature) 
texts

['丟進 冗員 堪稱 多餘 廚餘桶 慘遭 蝦球 貝斯手 超愛吃 超級 道菜 餐桌上 鳳梨',
 '一個月 一年 事情 可以 各位 大事 好像 完成 有點 沒有 甚麼 這樣 過去',
 '一下 不用 以前 名字 完要 微軟 放棄 是不是 沒記 瀏覽器 灌裝 發現 真的 確認 設為 還會 電腦 預設',
 '不屑 商業 地球 夾子 常常 木炭 洗腦 烤肉 烤肉醬 造成 陋習',
 '不管 動作 原意 嘻嘻 女生 寬子還 尊敬 帥哥 意思 成開 拉屎 攝影棚 暗喻 暗戀 有錢 漂亮 看上 笨笨 胖瘦 裡面 西裝 變帥 跑車 身上 那個 驚訝 高矮 高雅',
 '以前 半年 只靠 唸書 建中 有人 甚至 真的 考上 臺大 醫科 高中',
 '不料 人民 債務人 司法 司法院 在乎 大法官 尖尾 師孟 師孟說 情案 收賄 改革 法官 無罪 監委 監察權 脫產 豬吠 買票 赫然在目 週記 陳菊 鼎鼎大名',
 '不法 公義 告發 情事 按鈴 柯文哲 檢署 歡迎 蔡英文 證據 貪污',
 '出沒 喇牙 外面 天氣 太熱 家裡 專家 很少 抬頭 最好 最近 爬蟲類 異常 益蟲 看到 真的 蟑螂 解惑 蹤跡 進來 道歉 避暑 難免會 頻繁 餓死 驚怕',
 '使用 合法 嚴謹 字體 審查 康熙字典 彩繪 授權 活化 環境 石虎 管理處 觀光局 設計 車埕 重新 鐵道 集集',
 '中線 北平 和平統一 大陸 島上 幾架 戰機 措手不及 有史以來 海峽 海軍 澎湖 登陸 發起 破爛 第一站 而且 艦隊 解放 解放軍 防部 駐軍',
 '但用 挑毛病 歐韓 沒有 用心良苦 用詞 登陸戰 的確 精準 美國 美軍 臺灣 轟炸 風向',
 '傳說 別說 品牌 哪些 大多數 天才 接近 特斯拉 發展 發明 科學 窮途潦倒 紀念 電動汽車',
 '一直 不好 乳提 代表 個位 剛剛 壞人 小丑 甚麼 第一 肥肥 舉起 著舉 蠕動 角落 試刀 認命 路人 跳舞 這是 那種',
 '全力以赴 地方 執會 張嘉郡 徵召 換將 民主 民進黨 海線 淑莉林國賢 立委 臺灣 蘇治芬 許忠富 退選 選戰 鄉長 醜陋 雲林 麥寮',
 '一下 一樣 不好玩 你們 半夜 哦哦哦 哭哭 嗚嗚 怎麼辦 想問 決定 沒用 發廢文 發文 空虛 肥法 肥肥 說不定 路上 這裡 遇到 阿發',
 '一層 三層 便利商店 便當盒

In [8]:
# 刪除只出現一次的字詞，巨量文本分析，極少出現的字，代表不重要
frequency = defaultdict(int)
frequency
texts2 = []
for text in texts:
    if type(text) != float:
        texts2.append(text)
for text in texts2:
    text = text.split() #這個迴圈分切字詞
    for token in text:
        frequency[token] +=1  #這個迴圈是為了計算字數
texts_list = [[token for token in text.split() if frequency[token] >1 ] for text in texts2] #記住這個

In [9]:
# 文章數
len(texts_list)

4999

In [10]:
# 各篇文章文字袋
print (texts_list[0])

['丟進', '冗員', '堪稱', '多餘', '超級', '道菜', '鳳梨']


## 製作 dictionary

* 接著，將文字袋作「字典」(dictionary)轉換，取不重覆字詞集合。
* key為字詞本身，value為序號。注意指令為 corpora.Dictionary()，我們前面已匯入模組 from gensim import corpora。

In [11]:
# 引數 texts_list, 為各篇文章的文字袋 list 
dictionary = corpora.Dictionary(texts_list)
print(dictionary)

Dictionary(13622 unique tokens: ['丟進', '冗員', '堪稱', '多餘', '超級']...)


In [12]:
# 顯示dictionary 的字詞、編碼對應
dictionary.compactify()
print(dictionary.token2id)

{'丟進': 0, '冗員': 1, '堪稱': 2, '多餘': 3, '超級': 4, '道菜': 5, '鳳梨': 6, '一個月': 7, '一年': 8, '事情': 9, '可以': 10, '各位': 11, '大事': 12, '好像': 13, '完成': 14, '有點': 15, '沒有': 16, '甚麼': 17, '這樣': 18, '過去': 19, '一下': 20, '不用': 21, '以前': 22, '名字': 23, '微軟': 24, '放棄': 25, '是不是': 26, '沒記': 27, '瀏覽器': 28, '發現': 29, '真的': 30, '確認': 31, '還會': 32, '電腦': 33, '預設': 34, '不屑': 35, '商業': 36, '地球': 37, '常常': 38, '洗腦': 39, '烤肉': 40, '造成': 41, '陋習': 42, '不管': 43, '動作': 44, '原意': 45, '嘻嘻': 46, '女生': 47, '尊敬': 48, '帥哥': 49, '意思': 50, '拉屎': 51, '暗喻': 52, '暗戀': 53, '有錢': 54, '漂亮': 55, '看上': 56, '笨笨': 57, '胖瘦': 58, '裡面': 59, '西裝': 60, '跑車': 61, '身上': 62, '那個': 63, '驚訝': 64, '高矮': 65, '高雅': 66, '半年': 67, '唸書': 68, '建中': 69, '有人': 70, '甚至': 71, '考上': 72, '臺大': 73, '醫科': 74, '高中': 75, '不料': 76, '人民': 77, '司法': 78, '司法院': 79, '在乎': 80, '大法官': 81, '改革': 82, '法官': 83, '無罪': 84, '監委': 85, '監察權': 86, '買票': 87, '週記': 88, '陳菊': 89, '告發': 90, '柯文哲': 91, '檢署': 92, '歡迎': 93, '蔡英文': 94, '證據': 95, '貪污': 96, '出沒': 97, '外面': 98, '天氣': 99, '

In [13]:
gensim.corpora.dictionary.Dictionar

NameError: name 'gensim' is not defined

In [14]:
# 查某一個 vlue 的 key 值
ind=[i[0] for i in dictionary.items() if i[1]=='香港'][0]
print (ind)

493


## Corpus: term vector

## 製作 corpus 
* 由 texts_list 製作 corpus
* list of tuple, 每篇文章，每個字詞，出現的次數 

In [15]:
#將文字轉成向量
corpus = [dictionary.doc2bow(text) for text in texts_list]

In [16]:
print(len(corpus))
print (corpus[0])

4999
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]


## 儲存 dictionary, corpus

In [38]:
# 儲存字典。
dictionary.save(path+ '/dictionary.dict') 
# 儲存 corpus。
corpora.MmCorpus.serialize(path+ '/corpus.mm', corpus)  

# load舊的dictionary 與 corpus 檔

In [17]:
############################################################################
filename=path+ '/dictionary.dict'
corpusname=path+ '/corpus.mm'

# 暫存檔的位置
if os.path.isfile(filename):
    dictionary = corpora.Dictionary.load(filename)
    corpus = corpora.MmCorpus(corpusname)    
    print ( filename +', ' + corpusname)
else:
    print("can't find these two files")

C:/Users/HSIU/Desktop/108-1/python_ntu/dictionary.dict, C:/Users/HSIU/Desktop/108-1/python_ntu/corpus.mm


# Topics and Transformation

## Tfidf 轉換
* 在大多文章都有的字詞(例如介繫詞)，降低其權數，文章字數少者，增加其權數。

In [18]:
corpus

In [20]:
# 引數為 corpus
tfidf = models.TfidfModel(corpus, normalize=True) # step 1 -- initialize a model
# tfidf
corpus_tfidf = tfidf[corpus]
corpus_tfidf

In [21]:
print(len(corpus_tfidf))
corpus_tfidf[0]

4999


[(0, 0.4055356255118918),
 (1, 0.4055356255118918),
 (2, 0.3696075313534954),
 (3, 0.38451903770685103),
 (4, 0.27258587889104857),
 (5, 0.4055356255118918),
 (6, 0.38451903770685103)]

In [46]:
# 如果不加權 corpus
corpus[0]

[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0)]

In [ ]:
#練習到這裡

# 潛在語意分析，前(LSI) 已提及，此處只是用不同的模組重做一次
* TFIDF加權
* SVD 降維
* 萃取概念
* 最接近的字詞與文章


In [22]:
ntopics=200
# corpus_tfidf 為文字詞袋，經過 tfidf轉換, id2word 字詞的代碼與名稱
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=ntopics) 
# create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
corpus_lsi = lsi[corpus_tfidf] 

## 潛在語意分析，抽離出若干個概念，topic model

In [23]:
lsi.print_topics(ntopics)

[(0,
  '0.194*"八卦" + 0.177*"有沒有" + 0.130*"臺灣" + 0.125*"什麼" + 0.119*"怎麼" + 0.119*"大家" + 0.112*"是不是" + 0.106*"肥宅" + 0.099*"覺得" + 0.099*"各位"'),
 (1,
  '0.279*"八卦" + -0.276*"臺灣" + 0.261*"有沒有" + -0.200*"韓國瑜" + -0.191*"中國" + -0.190*"香港" + -0.182*"民進黨" + -0.164*"國民黨" + -0.158*"蔡英文" + -0.151*"柯文哲"'),
 (2,
  '-0.493*"有沒有" + -0.481*"八卦" + -0.165*"韓國瑜" + -0.139*"國民黨" + -0.128*"民進黨" + -0.104*"總統" + -0.099*"蔡英文" + -0.088*"柯文哲" + -0.085*"什麼" + -0.084*"立委"'),
 (3,
  '0.316*"臺灣" + 0.306*"中國" + 0.286*"香港" + -0.255*"韓國瑜" + 0.196*"美國" + -0.183*"國民黨" + -0.163*"柯文哲" + -0.156*"民進黨" + -0.139*"立委" + 0.136*"有沒有"'),
 (4,
  '0.217*"骯髒" + 0.216*"香水" + 0.216*"飛鷹" + 0.215*"仰望" + 0.215*"妝容" + 0.215*"錫箔" + 0.215*"目眩神迷" + 0.215*"粉末" + 0.215*"黑邊" + 0.215*"塗亮"'),
 (5,
  '0.264*"怎麼" + 0.212*"覺得" + 0.207*"大家" + -0.202*"肥宅" + 0.200*"知道" + 0.176*"有人" + 0.164*"還是" + 0.150*"真的" + 0.142*"不是" + 0.139*"出來"'),
 (6,
  '-0.297*"大家" + -0.221*"安安" + -0.206*"美國" + -0.189*"請問" + -0.169*"各位" + 0.148*"還是" + -0.145*"中國" + -0.141*"剛剛" + 0.

In [24]:
# 最前面的20個字詞
lsi.print_topic(0, topn=20)

'0.194*"八卦" + 0.177*"有沒有" + 0.130*"臺灣" + 0.125*"什麼" + 0.119*"怎麼" + 0.119*"大家" + 0.112*"是不是" + 0.106*"肥宅" + 0.099*"覺得" + 0.099*"各位" + 0.096*"一下" + 0.095*"香港" + 0.092*"最近" + 0.092*"知道" + 0.091*"這樣" + 0.091*"請問" + 0.088*"剛剛" + 0.087*"真的" + 0.087*"到底" + 0.087*"看到"'

In [25]:
# 字詞的權數，權數愈大表示在 topic 中愈重要。
# topic number=0, top number=5 
lsi.show_topic(0,topn=20)

[('八卦', 0.19407957907447895),
 ('有沒有', 0.1769961983335213),
 ('臺灣', 0.13015221062091445),
 ('什麼', 0.12463795186796732),
 ('怎麼', 0.11919303607798366),
 ('大家', 0.11893363472331801),
 ('是不是', 0.11190415491289758),
 ('肥宅', 0.10643989421873597),
 ('覺得', 0.09925506897281539),
 ('各位', 0.0986157887086374),
 ('一下', 0.09557622464848951),
 ('香港', 0.09468246186048533),
 ('最近', 0.0918079675685369),
 ('知道', 0.09157042593295528),
 ('這樣', 0.09141226557121213),
 ('請問', 0.09116069607771292),
 ('剛剛', 0.08760121452804545),
 ('真的', 0.08737946508849839),
 ('到底', 0.08718331505377089),
 ('看到', 0.0870179407222278)]

## 各主題代表的意義

In [25]:
for n in range(ntopics):
    keywords=[]
    for m in range(len(lsi.show_topic(n,topn=30))):
        keywords.append(lsi.show_topic(n,topn=30)[m][0])
    key=",".join(keywords)    
    print (n, key)

0 八卦,有沒有,臺灣,什麼,怎麼,大家,是不是,肥宅,覺得,各位,一下,香港,這樣,最近,知道,請問,剛剛,真的,到底,看到,好像,現在,安安,朋友,還是,有人,中國,今天,一個,如題
1 八卦,臺灣,有沒有,韓國瑜,中國,香港,民進黨,國民黨,蔡英文,柯文哲,總統,美國,中共,支持,立委,民眾,我們,肥宅,各位,大家,安安,民調,市長,最近,不分區,選舉,政黨,一下,剛剛,如題
2 有沒有,八卦,韓國瑜,國民黨,民進黨,總統,蔡英文,什麼,柯文哲,女生,立委,大家,現在,肥宅,可是,臺灣,民眾,覺得,請問,不分區,民調,支持,感覺,只是,香港,知道,最近,郭臺銘,那個,怎麼
3 臺灣,中國,香港,韓國瑜,國民黨,美國,民進黨,柯文哲,立委,總統,有沒有,不分區,民眾,中共,市長,民調,川普,蔡英文,郭臺銘,日本,政黨,八卦,我們,中國人,名單,參選,國家,臺灣人,支持,他們
4 骯髒,香水,只是,飛鷹,塗亮,妝容,手帕,散發,仰望,目眩神迷,粉末,錫箔,黑邊,指甲,氣味,一片,仍然,一條,背後,即使,可愛,有沒有,八卦,什麼,中國,商榷,韓國瑜,資源,一篇,美國
5 怎麼,覺得,肥宅,大家,知道,有人,還是,真的,不是,出來,應該,這樣,標題,到底,就是,各位,這麼,有沒有,八卦,柯文哲,韓國瑜,問卦,作者,這個,請問,不會,香港,很多,小弟,不過
6 大家,安安,還是,請問,各位,剛剛,一下,真的,美國,怎麼,帳號,有人,知道,中國,自己,問卦,川普,一堆,最近,時候,現在,比較,柯文哲,有沒有,突然,好奇,朋友,國民黨,八卦,文章
7 香港,美國,臺灣,中國,日本,川普,警察,抗爭,韓國瑜,他們,學生,總統,民眾,蔡英文,如果,民進黨,港警,支持,你們,反送中,今天,警方,立委,國家,經濟,是不是,市長,覺得,很多,什麼
8 大家,最近,是不是,好像,什麼,一下,這樣,還是,這個,安安,發現,肥宅,朋友,剛剛,但是,真的,時候,香港,一定,各位,臺灣,一直,好奇,美國,東西,很多,韓國瑜,到底,一樣,出來
9 還是,肥宅,今天,什麼,臺灣,標題,日本,問卦,知道,大家,作者,是不是,帳號,比較,到底,最近,發言,各位,文章,好像,現在,暫停,期限,理由,有沒有,很多,為天,超貼,一些,如果
10 韓國瑜,什麼,民進黨,市長,美國,香港,時代力量,政黨,總統,

93 不要,最近,突然,每次,各位,一直,直播,大概,有些,應該,這些,好像,就是,推薦,影片,就算,價格,多少,網友,時候,不過,一定,的話,政府,支持,可是,這是,加入,韓粉,國家
94 不用,一定,女友,你們,如何,人生,而已,媽媽,可以,聽說,其他,警察,世界,警方,下去,臺灣人,小孩,非常,辦法,那些,影片,不能,一些,什麼,記得,大概,畢業,多少,目前,臺北市
95 出來,看起來,鄉民,有點,聽說,小弟,幫忙,這是,怎麼辦,他們,一樣,其他,事情,今天,還是,常常,應該,影片,可能,我們,直播,一些,這麼,臺大,裡面,如題,開始,人生,上班,沒想到
96 一樣,政府,可是,突然,一天,每天,知道,常常,起來,事情,想到,很多,這是,辦法,國民黨,朋友,還有,幾乎,上面,剛剛,看過,這種,至少,有點,直接,每個,女友,那些,問題,肥宅
97 不要,應該,國家,這方面,工作,需要,這個,一天,不是,中華民國,不過,一點,支持,公司,沒有,不到,一直,老婆,或是,關係,可愛,韓粉,這麼,各位,直播,政治,安安,這樣,有點,一個
98 總統,已經,臺灣人,東西,一定,之前,結果,發生,嘻嘻,可能,地方,每個,多少,的話,小弟,裡面,人家,一樣,雖然,如何,中國人,員工,表示,政府,畢竟,一下,鄉民,那個,有點,颱風
99 不會,可是,臺北,以前,相信,民眾,好像,問題,選舉,已經,房子,後面,中共,嘻嘻,有沒有,這個,竟然,名字,女生,老婆,準備,感覺,小孩,一樣,自己,如題,這是,臺北市,法案,中國
100 有個,每次,正妹,老婆,警察,不要,明天,時間,不錯,而已,這種,比賽,運動,應該,臺灣人,嘻嘻,出門,女生,市長,一堆,所以,總統,只要,學生,還好,人家,得到,不是,看到,可愛
101 明明,中華民國,如果,裡面,出來,東西,結果,有點,整個,問題,還有,小弟,起來,其他,幫忙,直接,看過,立委,應該,安安,文章,標題,我們,一點,不用,好像,這麼,你們,臺大,聽說
102 而且,直接,中國人,幫忙,看起來,臺北市,這麼,高鐵,政治,其實,影片,之類,你們,大概,昨天,時候,聽說,一些,川普,臺中,鄉民,警方,別人,還在,只有,發生,韓粉,補助,臺大,看到
103 裡面,怎樣,其實,人家,你們,有個,而且,他們,根本,聽說,一天,工作,或是,常常,公司

186 新聞,之後,本魯,這是,容易,年輕,小時候,之類,人民,難道,人類,結果,不想,有點,老人,政治,好好,等等,多少,經濟,臺獨,只是,關心,選擇,道歉,前面,情況,中華民國,全部,品牌
187 一句,女友,網友,沒什麼,而是,這邊,我國,認識,加入,土地,只要,以為,差不多,文化,這麼,一起,垃圾,絕對,只會,有錢,完全,對方,或是,農地,聽說,已經,講話,幾個,藍色,平臺
188 繼續,價格,可能,等等,節目,那個,突然,下來,重點,哪個,英文,裡面,這種,奇怪,而且,不能,韓國,準備,同事,想要,機會,只有,價值,女友,發文,中文,文章,妹子,沒人,五毛
189 颱風,奇怪,萬元,道歉,剛好,人家,突然,一半,印度,女友,知道,一隻,邏輯,選舉,幾個,真是,已經,市場,使用,某些,補助,本魯,結婚,難道,還要,一定,孩子,不能,變成,臺中
190 老師,還好,媽媽,年輕人,等等,節目,想要,旁邊,名單,政策,好好,看看,每天,而已,客家人,終於,下去,平常,發生,經過,出門,大陸,好像,最強,醫生,男生,口味,網路,受到,代表
191 不用,法律,等等,一天,簡單,感謝,對方,女人,當時,老師,人民,發文,年輕,完整,蟑螂,中華民國,科技,而且,文章,畢竟,節目,怎樣,超級,進去,大概,居然,律師,認識,媽媽,成為
192 一些,剛好,有個,直接,而已,雖然,不是,過去,每次,反而,國際,對方,之類,已經,地方,中國人,機車,產品,需要,政策,高中,所以,裡面,行為,不到,幾個,明天,單純,館長,問題
193 韓粉,阿肥,下來,之前,如何,中華民國,不想,晚上,大陸,至少,目前,特別,其實,之類,床上,辦法,願意,臺北市,然後,就要,沒想到,繼續,以前,參加,不然,一點,旁邊,但是,有夠,活動
194 老師,照片,小妹,能力,其他,雖然,一種,而且,就算,韓國,某些,市場,人類,經濟,對方,不要,這些,分鐘,有點,推薦,提供,高雄,上班,便宜,噁心,小時候,小孩,還有,中午,運動
195 立委,結婚,看看,裡面,只要,臺北,直接,大學,妹妹,調查,活動,最後,大便,員工,看過,那些,重要,肥肥,明天,出國,有點,海外,蟑螂,就算,反觀,參選,醫生,願意,兩個,上班
196 一定,活動,以下,參選,進去,選舉,當然,民眾,有些,崩潰,中文,幾天,晚安,只要

# 左奇異值矩陣，字詞在topics 上的投影

In [58]:
# 左奇異值矩陣 row=字詞，col=topic
# 字詞在空間中的投影，沒有依大小排序
term_proj=lsi.projection.u
term_proj[0]

array([-8.16589536e-04, -2.50537097e-04, -1.18812895e-03, -1.03844569e-03,
        3.43211956e-04, -2.07890778e-03, -7.26875801e-04, -2.63424254e-04,
        1.25176635e-03,  1.21232118e-03, -1.02768694e-03, -1.69596863e-03,
        3.49229116e-04,  1.88355199e-03, -4.81819898e-06,  1.59256279e-03,
       -9.15560679e-04,  2.32729114e-03,  4.41331149e-04,  1.61440618e-03,
       -1.88791761e-03,  2.69796007e-03,  5.32459841e-04, -1.91327071e-03,
        1.13830016e-03,  2.60197927e-04,  2.82143002e-04, -5.72003469e-04,
        2.01327624e-03, -2.18722112e-03, -3.00657042e-03,  4.14750580e-04,
        4.70208354e-04, -2.98385988e-04, -6.86960459e-04,  1.58604010e-03,
        1.68892629e-03, -4.57958372e-04, -5.51006840e-04, -2.31783620e-03,
        1.98601799e-03, -4.85104033e-03,  3.99562424e-03, -1.64767016e-03,
        4.24777617e-03, -2.04864107e-03,  4.68547195e-04, -1.82432413e-03,
        3.03456129e-03, -1.46671768e-03, -3.05072339e-04,  7.71656187e-04,
       -7.48726529e-04,  

## 奇異值

In [27]:
# 奇異值 Singular Value
lsi.projection.s

array([4.51677892, 3.19245034, 2.75573181, 2.73433643, 2.65537055,
       2.40978453, 2.37809109, 2.3572071 , 2.33857232, 2.31190111,
       2.30390099, 2.28164679, 2.26582962, 2.24354381, 2.22450227,
       2.21623658, 2.20807376, 2.18700145, 2.17786194, 2.1543054 ,
       2.14833574, 2.12489928, 2.12040511, 2.11653236, 2.10588259,
       2.10048947, 2.09199184, 2.08322742, 2.07430678, 2.0708814 ,
       2.06613255, 2.05161911, 2.04683946, 2.0397838 , 2.03461324,
       2.02468961, 2.02015084, 2.01584338, 2.01312814, 2.00565993,
       1.99990525, 1.99501879, 1.98826286, 1.9858119 , 1.97594635,
       1.97320471, 1.96855127, 1.95820564, 1.94968346, 1.94831068,
       1.94669418, 1.9426796 , 1.93602095, 1.93154069, 1.92707853,
       1.9263909 , 1.91928277, 1.9166306 , 1.91009664, 1.90441391,
       1.90009466, 1.89723164, 1.89338896, 1.89159598, 1.88708044,
       1.8848264 , 1.87773799, 1.8706503 , 1.86932086, 1.86715361,
       1.8636206 , 1.86086972, 1.85664576, 1.85191977, 1.85053

# 右奇異值矩陣，文章在topics 上的投影

In [26]:
print(corpus_lsi[0])

[(0, -0.0070247960901723295), (1, 0.0027606043493368076), (2, -0.005155939468555436), (3, -0.0019859895057660415), (4, 0.0014122736659731255), (5, -0.009311625598778482), (6, -0.004050073758530697), (7, -0.003834712337566276), (8, 0.0043272744806262175), (9, 0.004079497062689267), (10, -0.006976589657111855), (11, -0.001911192279365355), (12, -0.00016658615590555022), (13, 0.008857677421396731), (14, -0.0016865196015182216), (15, 0.00894068268596972), (16, -0.005883076958179902), (17, 0.006737859595553648), (18, 0.002671931775668552), (19, 0.0018207456790518468), (20, -0.0011774306221347518), (21, 0.004546859063868449), (22, 0.0011232638131913146), (23, -0.0068428058514557605), (24, 0.0006860667383854718), (25, -0.006954156959676461), (26, 0.003390624330797952), (27, -0.004578554552441406), (28, 0.006275397880605893), (29, -0.009396136808349742), (30, -0.003793438346405959), (31, 0.0008595303178266317), (32, -0.008239995597047), (33, 0.0009739570943048338), (34, -0.0019574418044334043)

# 各篇文章在某一個概念上的投影的平均值，by date, 時間變化趨勢
* 注意有負值，因為係投影
* 依時間排序，可繪趨勢圖
* 日期 月份要改為數字，否則會排序錯誤

In [49]:
concept=1    ######################################選擇概念

date=list(set(list(df.date)))
ave_prj=[]
for d in date:
    sum=0
    num=0
    for n in range(len(df)):
        if df.date[n]==d:
            try:
                sum=sum+ corpus_lsi[n][concept][1]
                num=num+1
            except:     # 有些無值
                pass
    ave=sum/num
    ave_prj.append(ave)
data=list(zip(date,ave_prj))
df_prj=pd.DataFrame(data,columns=['date','prj'])

df_prj=df_prj.sort_index(by=['date'],ascending=True)
df_prj    
    

C:\Users\brat\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)


,date,prj
106,2019-Aug- 1,-0.014480
99,2019-Aug- 2,-0.005594
12,2019-Aug- 3,0.004825
42,2019-Aug- 4,-0.008862
101,2019-Aug- 5,-0.014557
108,2019-Aug- 6,-0.016102
36,2019-Aug- 7,0.001955
10,2019-Aug- 8,-0.000368
2,2019-Aug- 9,-0.003740
88,2019-Aug-10,-0.009548


## 相似矩陣

In [26]:
# transform corpus to LSI |space and index it
index = similarities.MatrixSimilarity(lsi[corpus_tfidf]) 
print (index)

MatrixSimilarity<4999 docs, 200 features>


## 出現關鍵字最多的文章

In [38]:
key='反送中'
rec=[]
count=[]
for n in range(len(df.feature)):
    c=df.feature[n].count(key)
    if c>0:
        rec.append(n)
        count.append(c)
print (rec)
print (count)

[356, 372, 447, 647, 1050, 1068, 1125, 1781, 1951, 2163, 2246, 2619, 2762, 2909, 2974, 3070, 3561, 3652, 3786, 3859, 3983, 4476, 4508, 4632, 4833, 4873]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## 文章與文章間的相似性

In [52]:
doc_list=list(df.title)

In [39]:
d=rec[0] 
d

356

In [53]:
# 任選一篇文件，在topics 空間上的投影
d=rec[0]   #################################
corpus_tfidf[d]
vec_lsi0 = lsi[corpus_tfidf[d]] # convert the query to LSI space

# print (doc_list[d])
# print(vec_lsi0)

# 該篇文章最接近的排列
sims = index[vec_lsi0] # perform a similarity query against the corpus
# 依最接近的排列
sims = sorted(enumerate(sims), key=lambda item: -item[1])
# print(sims) # print sorted (document number, similarity score) 2-tuples  

# 前十名
print (sims[0][0],doc_list[d])
print ('---------')
for n in range(1,20):
    print (sims[n][0],doc_list[int(sims[n][0])],'相似度: ',sims[n][1])

356 Re: [新聞] 陳同佳案》非常詭異！蘇貞昌︰絕不會陷入
---------
1337 [新聞] 相信陳同佳良心發現自首？ 呂秋遠「一個 相似度:  0.42942783
4426 [新聞] 黃之鋒抵達柏林 和德國外長交流 相似度:  0.39450482
4316 [爆卦] 透視中國 明教授發出粵語版緊急聲明 相似度:  0.38742653
1125 Re: [新聞] 反送中催化 移民台灣說明會場場爆滿 相似度:  0.38644737
3403 [爆卦] 拳王阿里前妻：LBJ無法跟阿里一樣偉大 相似度:  0.3846392
4929 Re: [問卦] 香港現在的民眾領袖是誰? 相似度:  0.38265687
3070 [新聞] 傳港府將撤回送中條例 港股狂歡暴衝近千 相似度:  0.38106722
2371 Re: [問卦] 其實中國想把香港炒成港獨吧? 相似度:  0.37763804
1050 [新聞] 印尼記者永久失明 將向港警務處長與警察提 相似度:  0.36942413
2974 [新聞] 不認同北京當局 港人發起雙十節插中華民國國旗 相似度:  0.3399132
1281 [問卦] 還來不及哭 穿過的子彈 就帶走 溫度 相似度:  0.3361845
1194 [新聞] 陳同佳案應來台受審？吳敦義：各種說法都 相似度:  0.33504143
2429 [新聞] 與黃之鋒對談　林昶佐感謝港人站在第一線 相似度:  0.33231458
3881 [問卦] 今日香港，明日台灣？ 相似度:  0.33152527
3983 [問卦] 林鄭月娥若問我該怎麼處理 要怎麼回答? 相似度:  0.3296862
4508 [問卦]  什麼時候可以入香港的市場？ 相似度:  0.32629248
3652 [問卦]香港反送中比太陽花及台灣所有社運猛？ 相似度:  0.3256274
4445 [新聞]台灣司法風暴！富商密宴筆記曝光　石木 相似度:  0.32301742
2089 [新聞] 港府：不會實施外匯管制 相似度:  0.32298082


## 文章叫出來看

In [54]:
print (df.title[d])
print ("****************************")
print (df.feature[d])
print ("****************************")
print (df.token[d])

Re: [新聞] 陳同佳案》非常詭異！蘇貞昌︰絕不會陷入
****************************
一直 人權 什麼 入境 再說 反送中 司法 國內 必要 拜託 明白 爭辯 真的 紅利 老共 自首 這件 陰謀 香港
****************************
獄送 說麼 為 什麼 一直 在 爭辯 送 不 送 中 醒醒 好 嗎 這是 一個 殺人案件 臺灣 口口聲聲 說多顧 人權 兩條 命 一個 媽媽 還有 肚子 裡的 孩子 被 殘忍 的 殺害 在 你 臺灣 發生 臺灣 居然 無法 替 冤死 的 人 做 主將 在 你 國內 漠視 你 法律 的 人 抓 起來 伏法 本來 就是 本分 你 需要 管 人家 說 了 什麼 嗎 很多 還在 扯 中共 的 陰謀 就當 它 是 可以 吧 是 陰謀 你 該 做 的 事 就 可以 不 做 就 好像 中共 說 每天 吃飯 就是 支持 一國兩制 你 就要 絕食 是 嗎 明明 不是 啊 每天 自己 該 做 的 事 還是 要 做 啊 為 什麼 要 一直 扯 是不是 老共 的 陰謀 你們 這些 一直 在 說 要 收 陳 男 的 是 呼應 港府 是 舔 共 是 的 人 拜託 醒 一下 這是 重大 刑事案件 你們 在 那邊 爭辯 老共 陰謀 時 是否 也 是 在 不顧 被害人 的 人權 只為 了 滿足 自己 反中仇 中 的 心態 什麼 叫 死 的 都 是 香港 人 所以 沒關係 在 你 國內 發生 的 擾亂 社會風氣 的 本來 就是 你 的 管轄 範圍 否則 法律 直接 寫 清楚 說 外國人 不受 此限 就 好 了 啊 還有 在 扯 引渡 司法 互助 的 拜託 搞清楚 一下 現在 火大 的 點是 假如 陳 男要 自首 為 什麼 你 要 拒絕 他 入境 拜託 別 再說 他 肯定 不是 自己 願意 自首 的 你 抓 你 的 你 為 什麼 要管 他 是不是 真的 要 自首 如果 明白 告訴 你 是 因為 共 收買 他 的 家人 說 你 去 自首 我 讓 你 全家 榮華富貴 我們 也 該 積極 讓 他 入境 逮捕 起來 啊 我 只是 覺得 臺灣 沒 必要 玩成 這樣 我 尊重 反中 的 人 不 喜歡 中國 的 情緒 但是 這件 事情 真的 沒 必要 弄成 一個 政治事件 吧 再說 小英 也 已經 靠 這件 事 收穫 頗豐 現在 只

# 字詞與字詞間的相似性

In [55]:
import numpy as np
def cos_simility(a, b):
    # list 改 array
    x = np.array(a)
    y = np.array(b)
    # 向量長度
    Lx=np.linalg.norm(x)
    Ly=np.linalg.norm(y)
    # cos 值
    cos=round(x.dot(y)/(Lx*Ly),8)
    return cos

# k 個數
# kw 關鍵字
# dictionary
# term_proj 字詞的投影
def kClosestTerms(k,kw,dictionary,term_proj): 
    index=[i[0] for i in dictionary.items() if i[1]==kw][0] 
    closestTerms = {}
    words=list(dictionary.items())
    terms=[]
    for n in words:
        terms.append(n[1])
    for i in range(len(terms)):
        if terms[i]!=kw:        
            closestTerms[terms[i]] = cos_simility(term_proj[index],term_proj[i])

    # 依 dictionary 的 value (即相似性) 排序
    sortedList = sorted(closestTerms , key= lambda l : closestTerms[l])

    # 取前 k個，字詞及相似性
    lis_sorted =sortedList[::-1][0:k]
    lis_sim=[]
    for l in lis_sorted:
        lis_sim.append(closestTerms[l])        

    data=list(zip(lis_sorted, lis_sim))   
    df=pd.DataFrame(data)      
    return df,data

In [59]:
kClosestTerms(20,"反送中",dictionary,term_proj)[0]

,0,1
0,連儂牆,0.512916
1,正義感,0.457892
2,撤回,0.456141
3,香港,0.434941
4,警務,0.434184
5,觀塘,0.434136
6,崩盤,0.431586
7,法會,0.415258
8,明講,0.407991
9,藝術家,0.407581


## 出現字詞的相關性

In [60]:
kw=['香港','反送中','港府','國民黨',"民進黨",'習近平']
index=[]
for k in range(len(kw)):
    index.append([i[0] for i in dictionary.items() if i[1]==kw[k]][0])
similar=[]
for n in range(len(index)):
    sim=[]
    for m in range(len(index)):           
        similarity= cos_simility(term_proj[index[n]],term_proj[index[m]]) 
        sim.append(similarity)
    similar.append(sim)
df_similar=pd.DataFrame(similar,columns=kw, index=kw)    
df_similar    

,香港,反送中,港府,國民黨,民進黨,習近平
香港,1.000000,0.434941,0.451570,0.013031,-0.006525,0.063961
反送中,0.434941,1.000000,0.382066,0.122598,0.008098,0.085598
港府,0.451570,0.382066,1.000000,-0.043585,-0.010977,0.124156
國民黨,0.013031,0.122598,-0.043585,1.000000,0.253485,0.014401
民進黨,-0.006525,0.008098,-0.010977,0.253485,1.000000,-0.147322
習近平,0.063961,0.085598,0.124156,0.014401,-0.147322,1.000000


* 相似性是兩向量的cos值，有可能是負值，表反向關係。 

## Latent Dirichlet Allocation, LDA
* LDA is a probabilistic extension of LSA (also called multinomial PCA), 
* so LDA’s topics can be interpreted as probability distributions over words. 
* These distributions are, just like with LSA, inferred automatically from a training corpus. 
* Documents are in turn interpreted as a (soft) mixture of these topics (again, just like with LSA).


## ntopics 個維度

## 加權

In [28]:
from gensim import models
ntopics=20 #######################################################################
# 給定 corpus, id2word, num of topics
lda = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=ntopics)
# 執行轉換
lda_res = lda[corpus_tfidf]
# 如果不加權
#lda_res = lda[corpus]

## 不加權

In [30]:
ntopics=20 #######################################################################
# 給定 corpus, id2word, num of topics
lda2 = models.LdaModel(corpus, id2word=dictionary, num_topics=ntopics)
# 執行轉換
lda_res2 = lda2[corpus]

In [31]:
print (type(lda_res2))
len(lda_res2)

<class 'gensim.interfaces.TransformedCorpus'>


4999

## 每篇文章在topic上的機率

In [32]:
# 經過lda轉換，在各 tops 的位置。
# 注意是機率的概念，均為正值 
for doc in lda_res:
    print(doc)

[(0, 0.014085456), (1, 0.014085456), (2, 0.014085456), (3, 0.014085456), (4, 0.014085456), (5, 0.014085456), (6, 0.014085456), (7, 0.014085456), (8, 0.014085456), (9, 0.014085456), (10, 0.014085456), (11, 0.014085456), (12, 0.014085456), (13, 0.73237634), (14, 0.014085456), (15, 0.014085456), (16, 0.014085456), (17, 0.014085456), (18, 0.014085456), (19, 0.014085456)]
[(0, 0.011019493), (1, 0.011019493), (2, 0.011019493), (3, 0.011019493), (4, 0.0110194925), (5, 0.0110194925), (6, 0.0110194925), (7, 0.0110194925), (8, 0.3356848), (9, 0.011019493), (10, 0.31186804), (11, 0.011019493), (12, 0.0110194925), (13, 0.0110194925), (14, 0.011019493), (15, 0.011019493), (16, 0.011019493), (17, 0.011019494), (18, 0.0110194925), (19, 0.16511574)]
[(0, 0.010838247), (1, 0.010838247), (2, 0.010838247), (3, 0.55967516), (4, 0.08297714), (5, 0.010838247), (6, 0.010838247), (7, 0.010838247), (8, 0.010838247), (9, 0.010838247), (10, 0.0108382525), (11, 0.010838247), (12, 0.010838248), (13, 0.010838248), 

[(0, 0.10125868), (3, 0.068944074), (6, 0.672625)]
[(0, 0.012679566), (1, 0.012679566), (2, 0.012679566), (3, 0.012679566), (4, 0.012679566), (5, 0.012679566), (6, 0.012679566), (7, 0.012679566), (8, 0.012679566), (9, 0.012679566), (10, 0.012679566), (11, 0.012679566), (12, 0.012679566), (13, 0.012679566), (14, 0.012679566), (15, 0.7590882), (16, 0.012679566), (17, 0.012679566), (18, 0.012679566), (19, 0.012679566)]
[(4, 0.11215297), (10, 0.6124779), (16, 0.119412065)]
[(0, 0.09147286), (2, 0.040067792), (16, 0.1779173), (19, 0.5429069)]
[(2, 0.091895476), (4, 0.16016436), (8, 0.03967437), (11, 0.5762093)]
[(7, 0.51976466), (8, 0.23818558), (16, 0.074697345)]
[(0, 0.011334156), (1, 0.011334156), (2, 0.062366616), (3, 0.011334156), (4, 0.011334156), (5, 0.011334156), (6, 0.011334156), (7, 0.011334156), (8, 0.011334156), (9, 0.011334156), (10, 0.011334157), (11, 0.011334156), (12, 0.011334157), (13, 0.011334156), (14, 0.5095032), (15, 0.23544951), (16, 0.011334157), (17, 0.011334156), (1

[(0, 0.011618548), (1, 0.011618548), (2, 0.47214583), (3, 0.14010742), (4, 0.011618548), (5, 0.011618548), (6, 0.011618548), (7, 0.011618548), (8, 0.011618548), (9, 0.086841576), (10, 0.011618548), (11, 0.115008466), (12, 0.011618548), (13, 0.011618548), (14, 0.011618548), (15, 0.011618548), (16, 0.011618548), (17, 0.011618548), (18, 0.011618548), (19, 0.011618548)]
[(3, 0.046654396), (10, 0.46225002), (15, 0.32557002)]
[(1, 0.83186746)]
[(0, 0.0111044375), (1, 0.011104438), (2, 0.0111044375), (3, 0.0111044375), (4, 0.7890157), (5, 0.0111044375), (6, 0.0111044375), (7, 0.0111044375), (8, 0.0111044375), (9, 0.0111044375), (10, 0.0111044375), (11, 0.0111044375), (12, 0.0111044375), (13, 0.0111044375), (14, 0.0111044375), (15, 0.0111044375), (16, 0.0111044375), (17, 0.0111044375), (18, 0.011104438), (19, 0.0111044375)]
[(0, 0.05160127), (17, 0.77072275)]
[(1, 0.45638236), (3, 0.041887745), (5, 0.042066235), (8, 0.15618531), (9, 0.04744148), (12, 0.09685568), (15, 0.042085335)]
[(0, 0.0107

[(1, 0.550227), (4, 0.033456076), (7, 0.09054874), (9, 0.03472976), (16, 0.16035786)]
[(0, 0.012106249), (1, 0.012106248), (2, 0.012106249), (3, 0.012106248), (4, 0.012106248), (5, 0.012106248), (6, 0.012106249), (7, 0.012106248), (8, 0.012106249), (9, 0.012106248), (10, 0.012106248), (11, 0.15136412), (12, 0.012106251), (13, 0.012106249), (14, 0.08248341), (15, 0.01210625), (16, 0.012106248), (17, 0.012106248), (18, 0.012106251), (19, 0.5603463)]
[(0, 0.010460276), (1, 0.010460272), (2, 0.010460272), (3, 0.010460273), (4, 0.010460276), (5, 0.010460272), (6, 0.010460272), (7, 0.010460272), (8, 0.010460272), (9, 0.010460272), (10, 0.20411539), (11, 0.6075997), (12, 0.010460272), (13, 0.010460272), (14, 0.010460272), (15, 0.010460272), (16, 0.010460276), (17, 0.010460272), (18, 0.010460272), (19, 0.010460272)]
[(0, 0.010426443), (1, 0.010426443), (2, 0.010426443), (3, 0.010426443), (4, 0.010426443), (5, 0.010426443), (6, 0.11177871), (7, 0.010426443), (8, 0.70054525), (9, 0.010426443), (

[(0, 0.01066861), (1, 0.01066861), (2, 0.01066861), (3, 0.010668612), (4, 0.79729635), (5, 0.01066861), (6, 0.010668615), (7, 0.010668613), (8, 0.010668613), (9, 0.01066861), (10, 0.01066861), (11, 0.010668611), (12, 0.01066861), (13, 0.01066861), (14, 0.01066861), (15, 0.010668611), (16, 0.010668611), (17, 0.010668611), (18, 0.01066861), (19, 0.010668611)]
[(16, 0.2226055), (17, 0.10936047), (18, 0.49838948)]
[(0, 0.010131557), (1, 0.010131557), (2, 0.010131557), (3, 0.010131557), (4, 0.010131557), (5, 0.010131557), (6, 0.05718177), (7, 0.7604502), (8, 0.010131557), (9, 0.010131557), (10, 0.010131557), (11, 0.010131557), (12, 0.010131557), (13, 0.010131557), (14, 0.010131557), (15, 0.010131557), (16, 0.010131557), (17, 0.010131557), (18, 0.010131557), (19, 0.010131557)]
[(0, 0.011638065), (1, 0.011638065), (2, 0.011638065), (3, 0.0116380695), (4, 0.011638065), (5, 0.011638065), (6, 0.011638065), (7, 0.011638065), (8, 0.011638065), (9, 0.011638065), (10, 0.77887684), (11, 0.011638065),

...

[(0, 0.12791325), (2, 0.12542345), (5, 0.16646698), (6, 0.05966998), (10, 0.17121385), (13, 0.09801287), (14, 0.07562765), (19, 0.056990866)]
[(2, 0.043025333), (6, 0.15028545), (8, 0.24064228), (10, 0.044325855), (13, 0.12761375), (15, 0.1666469), (17, 0.10713668)]
[(0, 0.01110261), (1, 0.01110261), (2, 0.06326681), (3, 0.064822644), (4, 0.01110261), (5, 0.13082431), (6, 0.011102611), (7, 0.01110261), (8, 0.01110261), (9, 0.01110261), (10, 0.3441703), (11, 0.01110261), (12, 0.01110261), (13, 0.01110261), (14, 0.011102612), (15, 0.17742054), (16, 0.01110261), (17, 0.01110261), (18, 0.01110261), (19, 0.064058796)]
[(0, 0.015855184), (1, 0.015855182), (2, 0.29492283), (3, 0.015855184), (4, 0.015855182), (5, 0.31837454), (6, 0.015855184), (7, 0.015855182), (8, 0.015855182), (9, 0.015855182), (10, 0.015855182), (11, 0.015855182), (12, 0.015855182), (13, 0.11716446), (14, 0.015855182), (15, 0.015855182), (16, 0.015855182), (17, 0.015855184), (18, 0.015855182), (19, 0.015855182)]
[(0, 0.1079

[(3, 0.1313397), (8, 0.18156968), (13, 0.102033146), (14, 0.1332126), (15, 0.2449175), (16, 0.06820374)]
[(0, 0.09539347), (1, 0.015608314), (2, 0.015608314), (3, 0.015608314), (4, 0.015608314), (5, 0.015608314), (6, 0.015608314), (7, 0.015608314), (8, 0.015608314), (9, 0.015608314), (10, 0.015608314), (11, 0.015608314), (12, 0.015608314), (13, 0.015608314), (14, 0.015608315), (15, 0.22254522), (16, 0.015608314), (17, 0.015608314), (18, 0.015608314), (19, 0.41671997)]
[(0, 0.0137307495), (1, 0.41040778), (2, 0.08646043), (3, 0.013730749), (4, 0.13801207), (5, 0.0137307495), (6, 0.013730749), (7, 0.013730749), (8, 0.013730749), (9, 0.013730749), (10, 0.013730749), (11, 0.013730749), (12, 0.013730749), (13, 0.013730749), (14, 0.0137307495), (15, 0.013730749), (16, 0.14542772), (17, 0.013730749), (18, 0.013730749), (19, 0.013730749)]
[(0, 0.011720124), (1, 0.011720124), (2, 0.011720125), (3, 0.011720125), (4, 0.011720125), (5, 0.011720125), (6, 0.011720124), (7, 0.011720124), (8, 0.011720

[(0, 0.012603644), (1, 0.012603644), (2, 0.012603644), (3, 0.012603644), (4, 0.012603644), (5, 0.012603644), (6, 0.10266592), (7, 0.012603644), (8, 0.012603644), (9, 0.012603644), (10, 0.2746273), (11, 0.012603646), (12, 0.012603644), (13, 0.012603644), (14, 0.012603644), (15, 0.1051739), (16, 0.31587455), (17, 0.012603644), (18, 0.012603644), (19, 0.012603644)]
[(0, 0.012575173), (1, 0.012575173), (2, 0.012575174), (3, 0.012575173), (4, 0.08572853), (5, 0.08754626), (6, 0.012575173), (7, 0.1812071), (8, 0.012575173), (9, 0.012575173), (10, 0.012575173), (11, 0.09667543), (12, 0.012575173), (13, 0.012575177), (14, 0.012575173), (15, 0.012575173), (16, 0.36021507), (17, 0.012575173), (18, 0.012575173), (19, 0.012575173)]
[(0, 0.0148567995), (1, 0.0148567995), (2, 0.10671623), (3, 0.0148567995), (4, 0.0148567995), (5, 0.11490868), (6, 0.10317782), (7, 0.20589405), (8, 0.0148567995), (9, 0.0148567995), (10, 0.0148567995), (11, 0.10259664), (12, 0.0148567995), (13, 0.0148567995), (14, 0.01

[(0, 0.01525807), (1, 0.01525807), (2, 0.01525807), (3, 0.01525807), (4, 0.01525807), (5, 0.01525807), (6, 0.01525807), (7, 0.01525807), (8, 0.01525807), (9, 0.01525807), (10, 0.01525807), (11, 0.01525807), (12, 0.01525807), (13, 0.15882331), (14, 0.01525807), (15, 0.01525807), (16, 0.5665314), (17, 0.01525807), (18, 0.01525807), (19, 0.01525807)]
[(0, 0.22397651), (1, 0.010693759), (2, 0.010693759), (3, 0.40376446), (4, 0.010693759), (5, 0.010693759), (6, 0.010693759), (7, 0.010693759), (8, 0.01069376), (9, 0.01069376), (10, 0.010693759), (11, 0.010693759), (12, 0.010693759), (13, 0.19046515), (14, 0.01069376), (15, 0.010693759), (16, 0.010693759), (17, 0.010693759), (18, 0.01069376), (19, 0.010693759)]
[(0, 0.011839091), (1, 0.01183909), (2, 0.01183909), (3, 0.01183909), (4, 0.01183909), (5, 0.01183909), (6, 0.01183909), (7, 0.01183909), (8, 0.4624597), (9, 0.01183909), (10, 0.01183909), (11, 0.01183909), (12, 0.01183909), (13, 0.01183909), (14, 0.01183909), (15, 0.01183909), (16, 0.

[(0, 0.011533734), (1, 0.24413043), (2, 0.21228343), (3, 0.011533733), (4, 0.1309892), (5, 0.011533733), (6, 0.011533733), (7, 0.011533734), (8, 0.011533733), (9, 0.08917347), (10, 0.0807271), (11, 0.011533733), (12, 0.011533733), (13, 0.011533734), (14, 0.011533733), (15, 0.011533733), (16, 0.08122411), (17, 0.011533733), (18, 0.011533734), (19, 0.011533734)]
[(0, 0.0114127705), (1, 0.0114127705), (2, 0.06786434), (3, 0.0114127705), (4, 0.13517605), (5, 0.21612465), (6, 0.0114127705), (7, 0.06568205), (8, 0.0114127705), (9, 0.0114127705), (10, 0.085446775), (11, 0.18294767), (12, 0.011412771), (13, 0.0114127705), (14, 0.0114127705), (15, 0.098392434), (16, 0.0114127705), (17, 0.0114127705), (18, 0.0114127705), (19, 0.0114127705)]
[(0, 0.017480712), (1, 0.017480712), (2, 0.017480712), (3, 0.36642691), (4, 0.017480712), (5, 0.017480712), (6, 0.017480712), (7, 0.017480712), (8, 0.017480712), (9, 0.017480712), (10, 0.017480712), (11, 0.017480712), (12, 0.017480712), (13, 0.3189203), (14, 

## show_topics()

In [33]:
# 顯示所有的 topics
top = lda.show_topics(20)
for t in top:
    print (t)

(0, '0.003*"各位" + 0.003*"事情" + 0.003*"就是" + 0.003*"是不是" + 0.002*"投資" + 0.002*"得到" + 0.002*"不夠" + 0.002*"好奇" + 0.002*"關係" + 0.002*"政權"')
(1, '0.003*"中華人民共和國" + 0.003*"地球" + 0.003*"餐廳" + 0.003*"以上" + 0.003*"行業" + 0.003*"有關" + 0.002*"不怕" + 0.002*"鏡頭" + 0.002*"曾經" + 0.002*"法國人"')
(2, '0.004*"感覺" + 0.003*"年月日" + 0.003*"隊長" + 0.003*"一律" + 0.003*"暴徒" + 0.002*"一定" + 0.002*"剛剛" + 0.002*"不過" + 0.002*"臺灣" + 0.002*"一整天"')
(3, '0.003*"服務" + 0.003*"員工" + 0.002*"分享" + 0.002*"早期" + 0.002*"金融" + 0.002*"承認" + 0.002*"起床" + 0.002*"上面" + 0.002*"賣場" + 0.002*"本身"')
(4, '0.003*"臺灣" + 0.003*"臺灣人" + 0.003*"全球" + 0.003*"任務" + 0.003*"夢想" + 0.003*"川普" + 0.002*"中國" + 0.002*"中共" + 0.002*"電影" + 0.002*"蘇聯"')
(5, '0.003*"瘋狂" + 0.002*"通過" + 0.002*"有沒有" + 0.002*"無數" + 0.002*"帝國" + 0.002*"女孩" + 0.002*"兩人" + 0.002*"地震" + 0.002*"知道" + 0.002*"單純"')
(6, '0.004*"時代力量" + 0.003*"選擇" + 0.003*"代理人" + 0.003*"民進黨" + 0.003*"臺北" + 0.003*"礦業法" + 0.003*"可是" + 0.003*"手段" + 0.002*"第四" + 0.002*"政黨"')
(7, '0.004*"高興" + 0.003*"英粉" + 0.003*"持

# show_topic() 顯示單一的 topic

In [34]:
# 第 n 個 topic
n=10
top = lda.show_topic(n,20)
for t in top:
    print (t)

('猶太人', 0.0037381526)
('胖子', 0.0026993412)
('安安', 0.002489678)
('男人', 0.0024105485)
('你們', 0.002370801)
('市議員', 0.0021853514)
('笑容', 0.0020956043)
('女同事', 0.002067465)
('一日', 0.0020120218)
('通過', 0.0019604873)
('原來', 0.0019312794)
('辦公室', 0.0019306281)
('焦慮', 0.001910661)
('睡覺', 0.0018791426)
('反中', 0.0018660307)
('要給', 0.0018492788)
('到處', 0.0018215678)
('萬一', 0.001808802)
('對方', 0.001777279)
('晚上', 0.0017737662)


In [35]:
ele=[]
for t in range(len(top)):
    ele.append(top[t][0])

## 所有 topics 的組成

In [36]:
select=range(ntopics)
for s in select:    
    top=lda.show_topic(s,30)
    ele=[]
    for t in range(len(top)):
        ele.append(top[t][0])   
    
    if s==select[0]:
        result=pd.DataFrame(ele,columns=[s])    
    else:         
        topics=pd.DataFrame(ele,columns=[s])       
        result = pd.concat([result, topics], axis=1)
result 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,各位,中華人民共和國,感覺,服務,臺灣,瘋狂,時代力量,高興,今天,板上,猶太人,自稱,離婚,如何,取得,或者,適合,師父,蔡英文,貓貓
1,事情,地球,年月日,員工,臺灣人,通過,選擇,英粉,今年,疑問,胖子,內政部,首歌,感謝,身為,中國,文章,名字,總統,被害人
2,就是,餐廳,隊長,分享,全球,有沒有,代理人,持續,攻擊,現實,安安,維持,美食,女孩兒,黃之鋒,貿易戰,很多,小姐,爸媽,低薪
3,是不是,以上,一律,早期,任務,無數,民進黨,紀元,還是,藍色,男人,任何,國民黨,從小,無誤,酒店,問卦,這樣,民調,看護
4,投資,行業,暴徒,金融,夢想,帝國,臺北,跑步,臺灣,而是,你們,相信,很少,道歉,贊成,隨便,兩個,窮人,年輕人,提告
5,得到,有關,一定,承認,川普,女孩,礦業法,垃圾,早就,人民幣,市議員,有加,身邊,有錢,捐款,臺灣,以後,對方,選民,版友
6,不夠,不怕,剛剛,起床,中國,兩人,可是,腦子,不會,縣市,笑容,有種,福利,基地,阿有,出口,哈哈哈,生氣,蔡英,世代
7,好奇,鏡頭,不過,上面,中共,地震,手段,美軍,只有,好萊塢,女同事,乳題,少女,騎著,聯合國,地方,批評,進步,建築,參加
8,關係,曾經,臺灣,賣場,電影,知道,第四,好處,德國,蝦皮,一日,火鍋,唸書,八卦,領先,帳號,小弟,今天,文說,政治人物
9,政權,法國人,一整天,本身,蘇聯,單純,政黨,喜歡,日本,調查,通過,父母,文組,喝完,香港,產品,實在,過去,代表,空屋


## 列出某個 tipic 機率達一個程度以上的貼文

In [37]:
n=10
prop=0.7
i=0
for d in range(len(lda_res)):
    try:
        for t in range(len(lda_res[d])):
            if lda_res[d][t][0]==n and lda_res[d][t][1]>prop:
                i+=1
                print (df.date.iloc[d],df.title.iloc[d] )    
    except:
        pass
print (i,'found')

2019-Sep- 5 Re: [問卦] 毒品這麼可怕 怎麼一堆敗類要用？
2019-Oct-14 [問卦] 問題點到底是殺小?
2019-Oct-16 Re: [問卦]吵死人的宮廟文化可以他媽的廢了嗎
2019-Nov-20 [問卦] 為什麼菜燕都要跟麻糬一起賣 ??
2019-Oct-28 Re: [問卦] 真的有人看過「人生走馬燈」嗎？
2019-Oct-10 [問卦] 談生意時講台語是不是很有氣勢
2019-Sep-26 [問卦] 長沢菜々香說這感覺已經不對了....
2019-Sep- 1 Re: [問卦] 跟foodpanda合作真的划算嗎？
2019-Oct- 3 Re: [問卦] 農地工廠並沒有就地合法 請勿聽信謠言…
2019-Nov-20 Re: [新聞] 直擊愛莉莎莎約洋男共度一晚實測！
2019-Aug- 2 [問卦] 慟！帝王麻痺斬斧太OP了吧(づ・ω・)づ
2019-Aug- 8 [問卦] 欸！舌頭靈活有什麼好處？
2019-Oct-10 [問卦] 澳門
2019-Aug-22 [問卦] 教室裡那台風琴叮咚叮咚叮嚀？
2019-Oct-10 [問卦] 現在偷放腿肉圖484沒人會發現 >//<
2019-Oct-14 [問卦] 對面的燒鴨店是不是不堪一擊?
2019-Nov- 1 Re: [問卦] 為何9.2愛吹捧中國貶低台灣但卻住台灣？
2019-Oct- 8 Re: [問卦] 女同事要我幫忙塗牙垢顯示劑
2019-Aug-16 [新聞] 罷韓行動遭打壓 民團怒衝韓國瑜市長室
2019-Aug-11 [新聞] 勞部周三開審議委員會 基本工資月薪估漲
2019-Oct-29 [問卦] 可愛的男生吃香還是帥的男生吃香
2019-Oct-27 Re: [問卦]  希特勒為何這麼痛恨猶太人
2019-Nov-20 Re: [問卦] 有沒有館長冰淇淋很佛心的卦？
2019-Aug-10 [新聞] 找上李進良做醫美？館長認「雙方有接觸」
2019-Nov-17 [新聞] 韓國瑜談國民黨未來：改革尚未成功、我
2019-Nov-24 [新聞] 特工竟是詐騙犯？韓國瑜促司法速查 真相
2019-Sep- 2 Re: [新聞] 胖到癱瘓、公開刺喉 逾5800人連署終止神豬比賽
2019-Aug- 1 [問卦] 見笑轉受氣？
2019-Aug-11 Re: 

# LDA 視覺呈現
* 須先安裝
* pip install pyldavis
* http://pyldavis.readthedocs.io/en/latest/readme.html#installation
* http://stat-graphics.org/movies/ldavis.html 很棒的範例影片

In [46]:
lda.show_topics()

[(6,
  '0.003*"吸引" + 0.003*"有人" + 0.003*"到底" + 0.003*"能夠" + 0.003*"有錢" + 0.003*"各位" + 0.003*"板上" + 0.003*"鄉民" + 0.002*"客人" + 0.002*"點閱"'),
 (0,
  '0.003*"兄弟" + 0.003*"選民" + 0.002*"颱風" + 0.002*"提醒" + 0.002*"一罐" + 0.002*"努力" + 0.002*"調查" + 0.002*"理由" + 0.002*"發言" + 0.002*"特地"'),
 (15,
  '0.003*"女孩" + 0.002*"首歌" + 0.002*"男人" + 0.002*"抬頭" + 0.002*"好奇" + 0.002*"餓死" + 0.002*"女生" + 0.002*"什麼" + 0.002*"本身" + 0.002*"垃圾"'),
 (11,
  '0.003*"師父" + 0.003*"藍綠" + 0.003*"很爽" + 0.003*"窮人" + 0.003*"粉絲" + 0.003*"業務" + 0.003*"還不" + 0.003*"進出" + 0.002*"蘇聯" + 0.002*"生氣"'),
 (18,
  '0.003*"英粉" + 0.003*"持續" + 0.003*"體驗" + 0.003*"朋友" + 0.002*"很難" + 0.002*"怎麼" + 0.002*"維持" + 0.002*"再說" + 0.002*"腦子" + 0.002*"劇情"'),
 (1,
  '0.003*"貿易戰" + 0.003*"臺灣" + 0.003*"早期" + 0.003*"出口" + 0.002*"經濟" + 0.002*"庶民" + 0.002*"離婚" + 0.002*"單純" + 0.002*"婚姻" + 0.002*"東西"'),
 (14,
  '0.003*"高興" + 0.003*"少女" + 0.003*"音樂" + 0.003*"人權" + 0.003*"一支" + 0.003*"晚上" + 0.003*"迷有" + 0.002*"疑問" + 0.002*"總統大選" + 0.002*"澳洲"'),
 (17,
  '0.003*"很大" 

In [47]:
import pyLDAvis.gensim
#pyLDAvis.gensim.prepare(lda, corpus_tfidf, dictionary)
pyLDAvis.display(pyLDAvis.gensim.prepare(lda, corpus_tfidf, dictionary))

C:\Users\brat\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [48]:
import pyLDAvis.gensim

pyLDAvis.display(pyLDAvis.gensim.prepare(lda2, corpus, dictionary))

# 作業
* 用你們自己的文本資料，比較 LSI 與 LDA 你覺得可以成功萃取出概念(或稱主題)嗎? 哪一個比較成功? 亦即比較容易識別?